In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline 

In [2]:
train = pd.read_csv('../apartment/train.csv')
#train.info()

In [3]:
train['CarPerArea']=train['GarageCars']/train['GarageArea']
train['CarPerArea']=train['CarPerArea'].fillna(0)

In [4]:
train['basementAnd1stFloorArea'] = train['1stFlrSF']+train['TotalBsmtSF']

In [5]:
train = train.drop('GarageArea',1)
train = train.drop('GarageCars',1)
train = train.drop('1stFlrSF',1)
train = train.drop('TotalBsmtSF',1)

In [6]:
from sklearn import preprocessing

catagorical_cols=[]
for column in train.columns.values:
    if (train[column].dtype=='O'):
        catagorical_cols.append(column)


for x in catagorical_cols:
    train[x]=train[x].fillna('None')
    
for x in catagorical_cols:
    label = preprocessing.LabelEncoder()
    train[x] = label.fit_transform(train[x])

In [7]:
from sklearn import preprocessing
from sklearn import linear_model, svm, gaussian_process
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Lasso
import numpy as np

In [8]:
train1 = train.drop('SalePrice',1)
X = train1
y1 =  pd.DataFrame(train['SalePrice'],columns=['SalePrice'])
y = pd.concat([ train['Id'],y1], axis=1)
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [9]:
for x in X_train.columns.values:
    if(X_train[x].isnull().sum() >0):
        X_train[x].fillna(X_train[x].mean(),inplace = True)
for y in X_test.columns.values:
    if(X_test[y].isnull().sum() >0):
        X_test[y].fillna(X_test[y].mean(),inplace = True)

D:\Anaconda3\lib\site-packages\pandas\core\generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [10]:
#feature selection

In [11]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(X_train)
sel.transform(X_test)

array([[8.930e+02, 2.000e+01, 7.000e+01, ..., 2.000e+00, 2.006e+03,
        2.127e+03],
       [1.106e+03, 6.000e+01, 9.800e+01, ..., 4.000e+00, 2.010e+03,
        2.963e+03],
       [4.140e+02, 3.000e+01, 5.600e+01, ..., 3.000e+00, 2.010e+03,
        2.036e+03],
       ...,
       [6.510e+02, 6.000e+01, 6.500e+01, ..., 5.000e+00, 2.008e+03,
        1.635e+03],
       [4.400e+02, 5.000e+01, 6.700e+01, ..., 8.000e+00, 2.009e+03,
        1.368e+03],
       [7.990e+02, 6.000e+01, 1.040e+02, ..., 7.000e+00, 2.009e+03,
        3.892e+03]])

In [14]:
X_train2 = X_train
y_train2 = y_train['SalePrice'].values

In [15]:
minmaxscaler = preprocessing.MinMaxScaler(copy=True, feature_range=(0, 1)).fit(X_train)
X_train = X_train.astype(np.float64)
y_train = y_train['SalePrice'].values.astype(np.float64)
X_train = minmaxscaler.fit_transform(X_train)

X_test = X_test.astype(np.float64)
y_test = y_test['SalePrice'].values.astype(np.float64)
X_test = minmaxscaler.transform(X_test)

In [1]:
clfs = {
        'svm':svm.SVR(), 
        'RandomForestRegressor':RandomForestRegressor(n_estimators=200),
        'LinearRegression':linear_model.LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
        'DecisionTreeRegressor':DecisionTreeRegressor(),
        'SGDRegressor':SGDRegressor(max_iter=5, tol = None),
        'GradientBoostingRegressor':GradientBoostingRegressor(),
        'MLPRegressor':MLPRegressor(solver='adam', activation = 'relu',hidden_layer_sizes=20, max_iter=1000, learning_rate='adaptive', random_state= None),
        'Lasso':Lasso(alpha = 0.1,max_iter=1000,random_state=None)
       }
for clf in clfs:
    try:
        clfs[clf].fit(X_train, y_train.ravel())   #why need to make this ravel()??
        y_pred = clfs[clf].predict(X_test)
        #print(clf + " cost:" + str(np.sum(y_pred-y_test)/len(y_pred)))
        #print(clf + " score:" + str(np.mean(np.absolute(y_pred-y_test))))
        #print(clf + " score:" + str(np.mean((y_pred-y_test)*(y_pred-y_test))))
        print(clf + " score:" + str(np.sqrt(np.mean((np.log(y_pred)-np.log(y_test))*(np.log(y_pred)-np.log(y_test))))))
    except Exception as e:
        print(clf + " Error:")
        print(str(e))

NameError: name 'svm' is not defined

In [15]:
#Prediction

In [16]:
test = pd.read_csv('../apartment/test.csv')
model = GradientBoostingRegressor()
from sklearn import preprocessing
label1 = preprocessing.LabelEncoder()

catagorical_cols1=[]
for column in test.columns.values:
    if (test[column].dtype=='O'):
        catagorical_cols1.append(column)


for x in catagorical_cols1:
    test[x]=test[x].fillna('None')
    
for x in catagorical_cols1:
    label1 = preprocessing.LabelEncoder()
    test[x] = label1.fit_transform(test[x])

In [17]:
for z in test.columns.values:
    if(test[z].isnull().sum() >0):
        test[z].fillna(test[z].mean(),inplace = True)

In [18]:
test['CarPerArea']=test['GarageCars']/test['GarageArea']
test['CarPerArea']=test['CarPerArea'].fillna(0)
test['basementAnd1stFloorArea'] = test['1stFlrSF']+test['TotalBsmtSF']
test = test.drop('GarageArea',1)
test = test.drop('GarageCars',1)
test = test.drop('1stFlrSF',1)
test = test.drop('TotalBsmtSF',1)

In [19]:
sel.transform(test)

array([[1461.,   20.,    3., ...,    9.,    4., 1778.],
       [1462.,   20.,    4., ...,    9.,    4., 2658.],
       [1463.,   60.,    4., ...,    9.,    4., 1856.],
       ...,
       [2917.,   20.,    4., ...,    9.,    0., 2448.],
       [2918.,   85.,    4., ...,    9.,    4., 1882.],
       [2919.,   60.,    4., ...,    9.,    4., 1992.]])

In [20]:
model.fit(X_train2,y_train2) 

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [21]:
X = test
y_predict = model.predict(X)

predicted_result = pd.DataFrame(data = y_predict,columns=['SalePrice'])
result = pd.concat([test['Id'],predicted_result],axis=1)

In [22]:
result.to_csv('../apartment/Feature_Selection_Version.csv',index = False)